In [34]:
import os
import nltk
import nltk.corpus
from nltk import sent_tokenize
from nltk import word_tokenize
from nltk.tag import StanfordNERTagger
from itertools import groupby
from nltk import word_tokenize, pos_tag, ne_chunk
from nltk import Tree
import re
import datetime

In [189]:
text = """Gary Winston Lineker was an excellent football player.
GARY WINSTON LINEKER was a striker.
gary winston lineker was born in England.
gARY WiNsTon lInEker is married to Danielle Bux.
Gary W. Lineker, Kanny Sansom and Peter Shilton played together.
The defenders:
    Gary Stevens
    02-02-1995
    Kenny Sansom
    Terry Butcher
    1994-02-03
The midfields were:
    - Bryan Robson;
    - Ray Wilkins;
    - Chris Waddle.
    1994-02-02"""

In [190]:
def get_date(text):
    continuous_chunk = {}
    date_reg_exp2 = re.compile(r'(\d{2}(/|-|\.)\w{3}(/|-|\.)\d{4})|([a-zA-Z]{3}\s\d{2}(,|-|\.|,)?\s\d{4})|(\d{2}(/|-|\.)\d{2}(/|-|\.)\d+)')
    i = 0
    for m in re.finditer(date_reg_exp2,text):
        continuous_chunk[i] = (m.group())
        i+=1
    return continuous_chunk

In [191]:
def get_Person(text):
    chunked = ne_chunk(nltk.pos_tag(word_tokenize(text)))
    continuous_chunk = {}
    i=0
    current_chunk = []
    for subtree in chunked:
        if type(subtree) == Tree and subtree.label() == 'PERSON':
            current_chunk.append(" ".join([token for token, pos in subtree.leaves()]))
        elif current_chunk:
            named_entity = " ".join(current_chunk)
            if named_entity not in continuous_chunk:
#                 continuous_chunk.append(named_entity)
                continuous_chunk[i]=named_entity
                i+=1
                current_chunk = []
        else:
            continue
        
    return continuous_chunk

In [192]:
def get_GPE(text):
    chunked = ne_chunk(nltk.pos_tag(word_tokenize(text)))
    continuous_chunk ={}
    i=0
    current_chunk = []
    for subtree in chunked:
            if type(subtree) == Tree and subtree.label() == 'GPE':
                for token, pos in subtree.leaves():
                    continuous_chunk[i]=token
                    i+=1
            elif current_chunk:
                named_entity = " ".join(current_chunk)
                if named_entity not in continuous_chunk:
                    continuous_chunk.append(named_entity)
                    current_chunk = []
            else:
                continue
    return continuous_chunk

In [195]:
a=get_GPE(text)
b=get_Person(text)
c=get_date(text)

In [204]:
fetch={}
if bool(a):
    fetch['GPE']=a
else:
    fetch['GPE']='NULL'
if bool(b):
    fetch['Name']=b
else:
    fetch['Name']='NULL'
if bool(c):
    fetch['Date']=c
else:
    fetch['Date']='NULL'

In [205]:
fetch

{'GPE': {0: 'England'},
 'Name': {0: 'Gary Winston Lineker',
  1: 'Danielle Bux',
  2: 'Gary W. Lineker',
  3: 'Kanny Sansom',
  4: 'Peter Shilton',
  5: 'Gary Stevens',
  6: 'Kenny Sansom Terry',
  7: 'Bryan Robson',
  8: 'Ray Wilkins',
  9: 'Chris Waddle'},
 'Date': {0: '02-02-1995', 1: '94-02-03', 2: '94-02-02'}}